In [3]:
import torch
import csv
import os
import random
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split
import pandas as pd
import sys  # Import the sys module
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import uuid
import itertools
import torch.nn as nn
import torch.nn.init as init

sys.path.append('../..')
from dataset_reader import Traces_Dataset

In [5]:
# load model RNN parameters
def rnn_add_dim(inputs, n_points=20, n_traces=16): 
    '''
    adding a dim to dataset, default = 2, so theres one channel for time, one for current
    '''
    return inputs.reshape(-1, n_traces, n_points).transpose(1,2)


traces_train_transformed = rnn_add_dim(traces_train)
traces_val_transformed = rnn_add_dim(traces_val)
traces_train_transformed.shape, traces_val_transformed.shape

NameError: name 'traces_train' is not defined

In [6]:
class simple_RNN_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, mlp_hidden_size, output_size):
        super(simple_RNN_Model, self).__init__()
        
        # Define the RNN layer (LSTM in this case)
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Define the MLP layers
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size + 9, mlp_hidden_size),
            nn.SiLU(),
            nn.Linear(mlp_hidden_size, output_size)
        )
        
    def forward(self, x):
        x_setup = x[:, :9]
        x_traces = x[:, 9:]
        # print(x_setup.shape, x_traces.shape)

        # Forward pass through the RNN layer
        _, (h_n, _) = self.rnn(rnn_add_dim(x_traces,20,16))
        # print(rnn_add_dim(x_traces,20,16).shape)
        # Extract the hidden state from the last time step
        rnn_output = h_n[-1]
        
        # Forward pass through the MLP layers
        # print(torch.cat((x_setup, rnn_output), dim=1).shape)
        output = self.mlp(torch.cat((x_setup, rnn_output), dim=1))
        
        return output

    def initialize_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)